<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Set-Up-Spark-Context" data-toc-modified-id="Set-Up-Spark-Context-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set Up Spark Context</a></span></li><li><span><a href="#Loading-and-Preprocessing-the-Example-Data" data-toc-modified-id="Loading-and-Preprocessing-the-Example-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Loading and Preprocessing the Example Data</a></span><ul class="toc-item"><li><span><a href="#Process-the-Features" data-toc-modified-id="Process-the-Features-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Process the Features</a></span></li></ul></li><li><span><a href="#Train-and-Predict-with-Random-Forest" data-toc-modified-id="Train-and-Predict-with-Random-Forest-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train and Predict with Random Forest</a></span></li><li><span><a href="#Evaluate-the-Model" data-toc-modified-id="Evaluate-the-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluate the Model</a></span></li><li><span><a href="#Using-Pipeline-and-Performing-a-Grid-Search-for-Optimal-Parameters" data-toc-modified-id="Using-Pipeline-and-Performing-a-Grid-Search-for-Optimal-Parameters-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Using Pipeline and Performing a Grid Search for Optimal Parameters</a></span><ul class="toc-item"><li><span><a href="#Evaluate-with-Cross-Validation-to-Find-Optimal-Model" data-toc-modified-id="Evaluate-with-Cross-Validation-to-Find-Optimal-Model-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Evaluate with Cross Validation to Find Optimal Model</a></span></li></ul></li></ul></div>

<a href="https://colab.research.google.com/github/flatiron-school/ds-spark/blob/main/spark-ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Run for Google Colab environment
!pip install pyspark==3.2.1
!apt install openjdk-8-jdk-headless -qq
!pip install mlflow

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

     |████████████████████████████████| 16.5 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 12.6 MB/s ta 0:00:01
     |████████████████████████████████| 147 kB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 25.0 MB 38.2 MB/s eta 0:00:01     |████████████████████████▌       | 19.1 MB 38.2 MB/s eta 0:00:01
     |████████████████████████████████| 182 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 60.1 MB/s eta 0:00:01     |███████████████████████▏        | 1.1 MB 60.1 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 19.3 MB/s eta 0:00:01
     |████████████████████████████████| 

     |████████████████████████████████| 2.4 MB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 25.5 MB 8.0 MB/s eta 0:00:011     |█████████████████████████████▉  | 23.7 MB 8.0 MB/s eta 0:00:01
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139100 sha256=279a2e073bbad61588b4d8e8b875d6b49238e9a728221df9db5d63d83d05ed16
  Stored in directory: /Users/andrewboucher/Library/Caches/pip/wheels/58/40/7c/d021d51dac18bfd095fb6837572ad2e6f1a34d221f4b1d976b
Successfully built databricks-cli
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 2.0.0
    Uninstalling importlib-metadata-2.0.0:
      Successfully uninstalled importlib-metadata-2.0.0
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.10
    Uninstalling url

In [2]:
import pyspark
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import feature
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

In [ ]:
# Get data directly from repo
!wget https://github.com/flatiron-school/ds-spark/releases/download/v1.0/US_births_2000-2014_SSA.csv

# Objectives

- Use `pyspark` to build machine learning models

# Set Up Spark Context

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# Loading and Preprocessing the Example Data

This example assumes that we have a holdout validation dataset somewhere else, so we don't need to perform a train-test split, we only need to perform cross validation

In [ ]:
# Load the file since we downloaded it earlie
df = spark.read.format('csv').option('header', 'true').\
load('US_births_2000-2014_SSA.csv')

In [ ]:
df.toPandas().head(3)

## Process the Features

In [ ]:
df.dtypes

In [ ]:
df = df.withColumn('births', df['births'].cast('int'))
df = df.withColumn('day_of_week', df['day_of_week'].cast('int'))
df = df.withColumn('date_of_month', df['date_of_month'].cast('int'))
df = df.withColumn('month', df['month'].cast('int'))
df = df.withColumn('year', df['year'].cast('int'))

In [ ]:
ohe = feature.OneHotEncoder(inputCols=['date_of_month',
                                                'day_of_week'],
                                     outputCols=['date_vec',
                                                  'day_vec'],
                                     dropLast=True)
one_hot_encoded = ohe.fit(df).transform(df)
one_hot_encoded.head()

Note the 'SparseVector' we've created!

In [ ]:
features = ['year', 'month', 'date_of_month', 'day_of_week']

target = 'births'

vector = VectorAssembler(inputCols=features, outputCol='features')
vectorized_df = vector.transform(one_hot_encoded)

The Vector Assembler is often what we want when we're building a model in Spark. [How does the VectorAssembler work?](https://spark.apache.org/docs/2.1.0/ml-features.html#vectorassembler)

In [ ]:
vectorized_df.columns

# Train and Predict with Random Forest

In [ ]:
rf_model = RandomForestRegressor(featuresCol='features',
                                 labelCol='births',
                                 predictionCol="prediction").fit(vectorized_df)

In [ ]:
predictions = rf_model.transform(vectorized_df).select("births", "prediction")
predictions.head(3)

# Evaluate the Model

Let's evaluate our model! [Here](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html) is a reference for the many metrics available in Spark.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='births')

evaluator.evaluate(predictions, {evaluator.metricName:"r2"})

In [ ]:
evaluator.evaluate(predictions, {evaluator.metricName:"mae"})

# Using Pipeline and Performing a Grid Search for Optimal Parameters

In [ ]:
one_hot_encoder = OneHotEncoder(inputCols=['date_of_month',
                                                'day_of_week'],
                                     outputCols=['date_vec',
                                                  'day_vec'],
                                     dropLast=True)
vector_assember = VectorAssembler(inputCols=features,
                                  outputCol='features')
random_forest = RandomForestRegressor(featuresCol='features',
                                      labelCol='births')
stages = [one_hot_encoder, vector_assember, random_forest]

pipeline = Pipeline(stages=stages)

Note: The stages in a pipeline can be either *Transformers* or *Estimators*. An estimator fits a DataFrame to produce a Transformer.

In [ ]:
random_forest.params

In [ ]:
params = ParamGridBuilder().addGrid(random_forest.numTrees, [20, 50, 100]).build()

In [ ]:
reg_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='births',
                                    metricName='mae')

## Evaluate with Cross Validation to Find Optimal Model

In [ ]:
cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=params,
    evaluator=reg_evaluator,
    parallelism=4
)

In [ ]:
cross_validated_model = cv.fit(df.cache())

In [ ]:
cross_validated_model.avgMetrics

In [ ]:
cross_validated_model.bestModel

In [ ]:
cross_validated_model.bestModel.stages

In [ ]:
cross_validated_model.bestModel.stages[2].getNumTrees